In [1]:
import os
import cv2
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
from keras.models import Sequential, load_model
from tensorflow.keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.applications.densenet import DenseNet121 as DenseNet121
from sklearn import preprocessing  #用來標準化刻度

#import matplotlib.pyplot as plt

cxr=pd.read_csv("cxr_control_mutli_ca_v2_single.csv")
count=0
train_x = [] 
test_x = [] 
train_y = [] 
test_y = [] 

for index, row in cxr.iterrows():
    if count<7132:
            img = cv2.imread("cxr_control_v2_mutli_ca" + "/" + row["dicom_id"]+".jpg")
         #  img = cv2.resize(img, (64, 64)) 
            train_x.append(img)
            train_y.append(row["eventV3"])
            count=count+1
    else :
        img = cv2.imread("cxr_control_v2_mutli_ca" + "/" + row["dicom_id"]+".jpg")
        # img = cv2.resize(img, (64, 64)) 
        test_x.append(img)
        test_y.append(row["eventV3"])
        count=count+1

train_x=np.array(train_x)
test_x=np.array(test_x)

train_y=np.array(train_y)
test_y=np.array(test_y)

#train_x = train_x.reshape(7305, 256, 256, 3)
#test_x = test_x.reshape(1502, 256, 256, 3)

train_x=train_x/255
test_x=test_x/255

print(train_x.shape)
print(test_x.shape)
#print(train_y)
#print(test_y)

(7132, 256, 256, 3)
(1452, 256, 256, 3)


In [2]:
def easy_cnn():
    easy_model = Sequential()
    easy_model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
    easy_model.add(BatchNormalization())
    easy_model.add(MaxPool2D((2, 2),padding='same'))
  #  easy_model.add(Activation('relu'))
    for i in range(3):
        easy_model.add(Conv2D(32, (3, 3)))
        easy_model.add(BatchNormalization())
        easy_model.add(MaxPool2D((2, 2),padding='same'))
  #      easy_model.add(Activation('relu'))
    easy_model.add(Flatten())
    easy_model.add(Dense(1,activation='sigmoid', kernel_initializer='random_uniform', bias_initializer='zeros'))
    easy_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=[tf.keras.metrics.BinaryAccuracy()])
    return easy_model

In [3]:
from keras.layers import Dense, LSTM, BatchNormalization
import tensorflow as tf
model=easy_cnn()

print(model.summary())
class_weight={0:1,1:10}
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history = model.fit(x=train_x, y=train_y, validation_split=0.2, epochs=10, batch_size=36, verbose=2,class_weight=class_weight)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9

In [4]:
neur_test_y_predicted = model.predict(train_x)
predict_test=[]
for i in range(neur_test_y_predicted.shape[0]): 
    if neur_test_y_predicted[i]>0.5:
        predict_test.append(1)
    else:
        predict_test.append(0)
predict_test = np.array(predict_test)

#print(predict_test)

from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,confusion_matrix

cm1 = confusion_matrix(train_y,predict_test)

sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])   #TPR
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])   #FPR

#print('Precision:',precision_score(y_test_1, predict_test))
#print('Recall:', recall_score(y_test_1, predict_test))
#print('f1-score: %f' % f1_score(y_test_1, predict_test))
print('Accuracy: %f' % accuracy_score(train_y, predict_test))
print('Sensitivity : %0.2f' % sensitivity )
print('Specificity :%0.2f' % specificity)

pd.crosstab(train_y,predict_test,rownames=['label'],colnames=['predict'])


Accuracy: 0.794448
Sensitivity : 0.07
Specificity :1.00


predict,0,1
label,,
0,5565,17
1,1449,101


In [5]:
neur_test_y_predicted = model.predict(test_x)
predict_test=[]
for i in range(neur_test_y_predicted.shape[0]): 
    if neur_test_y_predicted[i]>0.5:
        predict_test.append(1)
    else:
        predict_test.append(0)
predict_test = np.array(predict_test)

#print(predict_test)

from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,confusion_matrix

cm1 = confusion_matrix(test_y,predict_test)

sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])   #TPR
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])   #FPR

#print('Precision:',precision_score(y_test_1, predict_test))
#print('Recall:', recall_score(y_test_1, predict_test))
#print('f1-score: %f' % f1_score(y_test_1, predict_test))
print('Accuracy: %f' % accuracy_score(test_y, predict_test))
print('Sensitivity : %0.2f' % sensitivity )
print('Specificity :%0.2f' % specificity)

pd.crosstab(test_y,predict_test,rownames=['label'],colnames=['predict'])


Accuracy: 0.975207
Sensitivity : 0.00
Specificity :1.00


predict,0,1
label,,
0,1416,4
1,32,0
